In [1]:
import random
import numpy as np
import time
from tkinter import *

In [2]:
class Passenger(object):
    '''
    
    '''
    def __init__(self,enter_floor,target_floor,name):
        '''
        Args:
        enter_floor: The floor where passenger shows up and sends elevator request
        state: 'up' or 'down', intended moving direction of the passenger  
        waiting_time: Time tracker of the accumulated waiting time, default to 0
        '''
        self.enter_floor=enter_floor
        self.target_floor=target_floor
        self.name=name
        if enter_floor<target_floor:
            self.state='up'
        elif enter_floor>target_floor:
            self.state='down'
        else:
            raise ValueError('Passenger {} does not need elevatpr.'.format(name) )
        self.waiting_time=0
    
    def getName(self):
        return self.name

    def getState(self):
        return self.state
    
    def getEnterFloor(self):
        return self.enter_floor
    
    def getTargetFloor(self):
        return self.target_floor
    
    def getWaitingTime(self):
        return self.waiting_time
    
    def updateWaitingTime(self):
        self.waiting_time=self.getWaitingTime()+1


In [3]:
class ElevatorTarget(object):
    def __init__(self,floor,state):
        self.floor=floor
        self.state=state
        
    def __str__(self):
        return '- ({},{})'.format(self.floor,self.state)
        
    def getFloor(self):
        return self.floor
    
    def getState(self):
        return self.state
    
    def getDistance(self,end,tall):
        '''
        end: another ElevatorTarget object
        tall: int, the height of building, max_floor-min_floor
        '''
    
    def distance(self,end,min_floor,max_floor):
        if self.getState()=='up':
            if end.getState()=='up':
                if self.getFloor()<=end.getFloor():
                    return  end.getFloor()-self.getFloor()
                else:
                    return (max_floor-min_floor)*2-self.getFloor()+end.getFloor()
                
            elif end.getState()=='down':
                return max_floor*2-self.getFloor()-end.getFloor()

            elif end.getState()==None:
                if self.getFloor()<=end.getFloor():
                    return  end.getFloor()-self.getFloor()
                else:
                    return max_floor*2-self.getFloor()-end.getFloor()

        elif self.getState()=='down':
            if end.getState()=='up':
                return self.getFloor()+end.getFloor()-min_floor*2
            
            elif end.getState()=='down':
                if self.getFloor()>=end.getFloor():
                    return self.getFloor()-end.getFloor()
                else:
                    return (max_floor-min_floor)*2+self.getFloor()-end.getFloor()
                
            elif end.getState()==None:
                if self.getFloor()>=end.getFloor():
                    return  self.getFloor()-end.getFloor()
                else:
                    return self.getFloor()+end.getFloor()-min_floor*2
        
        elif self.getState()==None:
            return abs(end.getFloor()-self.getFloor())
        
    

In [4]:
class BasicElevator(object):
    '''
    
    '''
    def __init__(self,min_floor,max_floor):
        '''
        '''
        self.max_floor=max_floor
        self.min_floor=min_floor
        self.current_floor=min_floor
        self.state=None
        self.target_list=[]
        self.travel_distance=0
    
    def getMaxFloor(self):
        return self.max_floor
    
    def getMinFloor(self):
        return self.min_floor
    
    def getState(self):
        return self.state
    
    def setState(self,new_state):
        '''
        Args:
        new_state: string 'up','down' or None when standby
        '''
        self.state=new_state
    
    def getCurrentFloor(self):
        return self.current_floor
    
    def updateCurrentFloor(self):
        if self.getState()=='up':
            if self.getCurrentFloor()==self.getMaxFloor():
                raise ValueError('Exceed max floor')    
            else: self.current_floor=self.getCurrentFloor()+1
                
        if self.getState()=='down':
            if self.getCurrentFloor()==self.getMinFloor():
                raise ValueError('Exceed min floor')    
            else: self.current_floor=self.getCurrentFloor()-1
    
    def getTargetList(self):
        return self.target_list
    
    def setTargetList(self,new_list):
        self.target_list=new_list
    
    def checkTargetList(self):
        '''
        eliminate reached target from top of the target list
        '''
        old_list=self.getTargetList()
        current_floor=self.getCurrentFloor()
        current_state=self.getState()
        
        if old_list==[]:
            pass
        
        elif old_list[0].getFloor()==current_floor:
            self.setTargetList(old_list.copy()[1:])
            if old_list[0].getState()!=None:
                self.setState(old_list[0].getState())
        
        if self.getTargetList()==[]:
            self.setState(None)
                
    def addTargetList(self,new_target):
        '''
        Args:
        new_target: a ElevatorTarget object
        '''
        old_list=self.getTargetList()
        current_state=ElevatorTarget(self.getCurrentFloor(),self.getState())
        min_floor=self.getMinFloor()
        max_floor=self.getMaxFloor()
            
        if new_target.getFloor() not in [target.getFloor() for target in old_list]:
            old_list.append(new_target)
            new_list=[old_list[i] for i in np.argsort([current_state.distance(i,min_floor,max_floor) for i in old_list])]
            self.setTargetList(new_list)
            
        else:
            new_list=[]
            for target in old_list:
                if target.getFloor()==new_target.getFloor():
                    if target.getState()==None:
                        new_list.append(new_target)
                    elif new_target.getState()==None:
                        new_list.append(target)
                    elif new_target.getState()==target.getState():
                        new_list.append(target)
                    else:
                        new_list.append(target)
                        new_list.append(new_target)
                else:
                    new_list.append(target)
                    
            ordered_list=[new_list[i] for i in np.argsort([current_state.distance(i,min_floor,max_floor) for i in new_list])]
            self.setTargetList(ordered_list)
                            
    def passengerTakeOff(self,current_passenger):
        '''
        Args:
        current_passenger: a list of passengers in this elevator'''
        new_passenger=[]
        tookoff_passenger=[]
        for passenger in current_passenger:
            if passenger.getTargetFloor()==self.getCurrentFloor():
                #print('- Passenger took off:',passenger.getName())
                #print('- Total waiting time:',passenger.getWaitingTime())
                #print(passenger.getName(),passenger.getWaitingTime())
                tookoff_passenger.append(passenger)
            else:
                new_passenger.append(passenger)
        return (new_passenger,tookoff_passenger)
    
    def passengerTakeOn(self,waiting_passenger):
        '''
        Args:
        waiting_passenger: a dictionary with floor as keys and list of passengers as values
        
        Returns:
        a tuple (list of new passenger,dictionary of new waiting passenger)
        '''
        new_waiting_passenger=[]
        new_takeon_passenger=[]
        floor=self.getCurrentFloor()
        
        if self.getTargetList()==[]:
            return ([],waiting_passenger)
            
        elif floor!=self.getTargetList()[0].getFloor():
            return ([],waiting_passenger)
        
        elif waiting_passenger[floor]==[]:
            return ([],waiting_passenger)
        
        else:
            for passenger in waiting_passenger[floor]:
                if passenger.getState()==self.getTargetList()[0].getState():
                    new_takeon_passenger.append(passenger)
                    #print('- Passenger took on:',passenger.getName())
                    #print('- Target:',passenger.getTargetFloor())
                else:
                    new_waiting_passenger.append(passenger)              
            waiting_passenger[floor]=new_waiting_passenger
            return (new_takeon_passenger,waiting_passenger)
        
    def updateState(self):
        if self.getTargetList()==[]:
            pass
        else:
            next_target=self.getTargetList()[0]
            current_floor=self.getCurrentFloor()
            if current_floor>next_target.getFloor():
                self.setState('down')
            elif current_floor<next_target.getFloor():
                self.setState('up')
            else:
                self.setState(next_target.getState())
    
    def getDistance(self):
        return self.travel_distance
    
    def updateDistance(self):
        if self.getState()!=None:
            self.travel_distance=self.getDistance()+1    

In [5]:
def timesteps(n,elevator):
    min_floor=elevator.getMinFloor()
    max_floor=elevator.getMaxFloor()
    counter=0
    waiting_passenger={}
    for i in range(min_floor,max_floor+1):
        waiting_passenger[i]=[]
    passenger_1=[]
    
    # Set up canvas constants
    master=Tk()
    w = Canvas(master, width=1000, height=520)
    w.pack()
    w.create_rectangle(70, 10, 170, 510, fill = "white")
    
    for i in range(1,10):
        w.create_line(70, 50*i+10, 170, 50*i+10)
        
    for i in range(10):
        w.create_text(20, 50*i+18,anchor=NW,text=str(10-i)+'F',font = ('TimesNewRoman',15))
        
    w.create_text(480, 30, anchor=NW,text='Passengers take off: ')
    w.create_text(480, 150, anchor=NW,text='Passengers take on: ')
    w.create_text(480, 270, anchor=NW,text='Passengers in the elevator: ')
    
    master.update()
    
    for i in range(n):
        #print('Timestep -',i)
        timestep_label=w.create_text(480, 10, anchor=NW,text='Timestep '+str(i))
        
        # update floor
        elevator.updateCurrentFloor()
        #print('Current Floor:',elevator.getCurrentFloor())
        
        # Draw empty elevator
        floor=elevator.getCurrentFloor()
        elevator_sqr=w.create_rectangle(72, (10-floor)*50+12, 168, (10-floor)*50+58, fill = "grey")
        
        # update travel distance for KPI
        elevator.updateDistance()

        # Draw waiting passengers - 1:
        draw_passenger_wait=[]
        text_passenger_wait=[]
        for f in waiting_passenger:
            for i,p in enumerate(waiting_passenger[f]):
                row=i//3
                col=i%3
                draw_passenger_wait.append(w.create_rectangle(172+col*100, 
                                                              (10-f)*50+12+row*25, 
                                                              192+col*100,
                                                              (10-f)*50+32+row*25, 
                                                              fill = "red"))
                text_passenger_wait.append(w.create_text(200+col*100,
                                                         (10-f)*50+12+row*25,
                                                         anchor=NW,
                                                         text=p.getName()+'-'+p.getState()))
        # Draw passenger in the elevator - 1:
        draw_passenger_on=[]
        text_passenger_on=[]
        for i,p in enumerate(passenger_1):
            row=i//4
            col=i%4
            draw_passenger_on.append(w.create_rectangle(75+col*10,
                                                        (10-floor)*50+15+row*10,
                                                        85+col*10,
                                                        (10-floor)*50+25+row*10,
                                                        fill = 'red'))
            text_passenger_on.append(w.create_text(480, 
                                           290+20*i, 
                                           anchor=NW,
                                           text=p.getName()+' heading to '+str(p.getTargetFloor())+'F'))
        master.update()
        time.sleep(1)
        
        # Let passengers who reach the target take off
        passenger_1,tookoff_passenger=elevator.passengerTakeOff(passenger_1)
        
        # Draw off passenger
        for p in draw_passenger_on:
            w.delete(p)
        for p in text_passenger_on:
            w.delete(p)
            
        draw_passenger_off=[]
        
        for i,p in enumerate(tookoff_passenger):
            draw_passenger_off.append(w.create_text(480, 
                                                    50+20*i, 
                                                    anchor=NW,
                                                    text=p.getName()+' waited for '+str(p.getWaitingTime())+' s'))
        
        # Draw passenger in the elevator - 2
        draw_passenger_on=[]
        text_passenger_on=[]
        for i,p in enumerate(passenger_1):
            row=i//4
            col=i%4
            draw_passenger_on.append(w.create_rectangle(75+col*10,
                                                        (10-floor)*50+15+row*10,
                                                        85+col*10,
                                                        (10-floor)*50+25+row*10,fill = 'red'))
            text_passenger_on.append(w.create_text(480, 
                                                   290+20*i, 
                                                   anchor=NW,
                                                   text=p.getName()+' heading to '+str(p.getTargetFloor())+'F'))
            
        master.update()
        time.sleep(1)
        
        # Take new passengers
        new_takeon,waiting_passenger=elevator.passengerTakeOn(waiting_passenger)
                
        # Draw passenger in the elevator - 3
        new_target_text=[]
        for i,p in enumerate(new_takeon):
            row=(i+len(passenger_1))//4
            col=(i+len(passenger_1))%4
            draw_passenger_on.append(w.create_rectangle(75+col*10,
                                                        (10-floor)*50+15+row*10,
                                                        85+col*10,
                                                        (10-floor)*50+25+row*10,
                                                        fill = 'red'))
            new_target_text.append(w.create_text(480, 
                                                 170+20*i, 
                                                 anchor=NW,
                                                 text=p.getName()+' heading to '+str(p.getTargetFloor())+'F'))
            text_passenger_on.append(w.create_text(480, 
                                                   290+20*(i+len(passenger_1)), 
                                                   anchor=NW,
                                                   text=p.getName()+' heading to '+str(p.getTargetFloor())+'F'))
        passenger_1=passenger_1+new_takeon
        
        # Checked current target
        elevator.checkTargetList()
        
        # Receive new target orders
        for new_passenger in new_takeon:
            elevator.addTargetList(ElevatorTarget(new_passenger.getTargetFloor(),None))
        
        # Allocate new waiting orders
        if random.random()>0.5:
            enter_floor=int(min_floor+np.floor(random.random()*(max_floor-min_floor)))
            target_floor=int(min_floor+np.floor(random.random()*(max_floor-min_floor)))
            if enter_floor!=target_floor:
                name='p'+str(counter)
                counter+=1
                person=Passenger(enter_floor,target_floor,name)
                elevator.addTargetList(ElevatorTarget(person.getEnterFloor(),person.getState()))
                waiting_passenger[enter_floor]=waiting_passenger[enter_floor]+[person]
                #print('New passenger waiting, name: {},floor: {},directon: {}'.format(person.name,person.getEnterFloor(),person.getState()))
        
        # Draw waiting passengers - 2
        for p in draw_passenger_wait:
            w.delete(p)    
        for p in text_passenger_wait:
            w.delete(p)
            
        draw_passenger_wait=[]
        text_passenger_wait=[]
        for f in waiting_passenger:
            for i,p in enumerate(waiting_passenger[f]):
                row=i//3
                col=i%3
                draw_passenger_wait.append(w.create_rectangle(172+col*100, 
                                                              (10-f)*50+12+row*25, 
                                                              192+col*100,
                                                              (10-f)*50+32+row*25, 
                                                              fill = "red"))
                text_passenger_wait.append(w.create_text(200+col*100,
                                                         (10-f)*50+12+row*25,
                                                         anchor=NW,
                                                         text=p.getName()+'-'+p.getState()))
        master.update()
        time.sleep(1)
        #print('Current targets:')
        #for i in elevator.getTargetList():
            #print(i)

        # Update elevator moving direction
        elevator.updateState()
        #print('Current state:',elevator.getState())
        
        # Update passenger waiting time
        for f in waiting_passenger:
            temp=[]
            for passenger in waiting_passenger[f]:
                passenger.updateWaitingTime()
                temp.append(passenger)
            waiting_passenger[f]=temp
        #print('Waiting list:',[[j.getName()+'-'+j.getState() for j in waiting_passenger[i]] for i in waiting_passenger])        
        
        #print('------')

        
        # Clear Canvas:
        w.delete(timestep_label)
        w.delete(elevator_sqr)
        for p in draw_passenger_on:
            w.delete(p)
        
        for p in draw_passenger_off:
            w.delete(p)
        
        for p in new_target_text:
            w.delete(p)
        
        for p in draw_passenger_wait:
            w.delete(p)
            
        for p in text_passenger_wait:
            w.delete(p)
        
        for p in text_passenger_on:
            w.delete(p)
                                    
        
    mainloop()

In [6]:
elevator_1=BasicElevator(1,10)

- 去掉方框的框线
- 增加下电梯的效果
- 修改右侧文字大小
- 添加交互按钮？

In [ ]:
timesteps(50,elevator_1)